# Predecir si el ejercicio es correcto
Aplicar Machine Learning para predecir si el paciente está realizando correctamente el ejercicio (clasificación binaria)

In [ ]:
# ---------- importar librerías---------
# Manipular los datos
import pandas as pd

# Gráficas
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
sns.set_theme(style="darkgrid")

# scikit-learn (ML en python)
## Procesar el dataset
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import LeaveOneGroupOut
# or LeavePGroupsOut
## Modelos ML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
## Evaluación de los modelos
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
## Hiperparametrizacion
from sklearn.model_selection import RandomizedSearchCV

# Class imbalance
from imblearn.under_sampling import NearMiss, EditedNearestNeighbours
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN
from collections import Counter

## Seleccion de variables
from sklearn.decomposition import PCA

# Pipeline
from imblearn.pipeline import Pipeline


# Para ignorar los Warnings
import warnings
from sklearn.exceptions import FitFailedWarning 

warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)

In [ ]:
#---------Importar los datos-----------
# Dataframe medidas calculadas por repetición
df = pd.read_csv('../csvFiles/medidasPerRepetition.csv', dtype=object)

In [ ]:
# ------Preparar el dataset-------
encoder = OrdinalEncoder(categories=[list(set(df["Position"].values))])
encoder.fit(df[["Position"]])
df["Position"] = encoder.transform(df[["Position"]])

# pasar variable obj to numeric
df = df.apply(pd.to_numeric, errors='ignore')

# cmabiar las poorly executed to incorrectly executed
df[df['CorrectLabel']==3] = 2

## ML simple

In [ ]:
#-------Funciones--------

# Para entrenar los modelos de ML
def aplicar_modelos(X: pd.DataFrame, y: pd.DataFrame, file):
    """
    Aplica una lista de modelos de aprendizaje automático a los datos de prueba y muestra
    los informes de evaluación.

    Parámetros:
    * X: datos de entrenamiento.
    * y: target de entrenamiento.
    * file: archivo para escribir los resultados.
    """
    # creación de los modelos
    knn = KNeighborsClassifier()  # K-Neighbors classifier
    dtree = DecisionTreeClassifier()  # Decision tree
    nb = GaussianNB()  # Naive Bayes
    svm = SVC()  # Support Vector Machine
    rf = RandomForestClassifier()  # Random Forest
    lr = LogisticRegression()  # Logistic Regression

    models = [knn, dtree, nb, svm, rf, lr]
    logo = LeaveOneGroupOut()
    grupos = X['SubjectID']

    # Iteramos sobre los modelos
    for modelo in models:
        try:
            accuracy_scores = cross_val_score(modelo, X, y, groups=grupos, cv=logo, scoring='accuracy')
            recall_scores = cross_val_score(modelo, X, y, groups=grupos, cv=logo, scoring='recall')
            file.write(f"\n\t Modelo: {modelo.__class__.__name__}\n")
            file.write("\t%0.2f accuracy with a standard deviation of %0.2f\n" % (accuracy_scores.mean(), accuracy_scores.std()))
            file.write("\t%0.2f recall with a standard deviation of %0.2f\n" % (recall_scores.mean(), recall_scores.std()))            
        except ValueError as e:
            file.write(f"Skipping {modelo.__class__.__name__} due to ERROR: {e}\n")

# Aplicar imbalance learn
def aplicar_imblearn(X, y, file):
    nm1 = NearMiss(version=1, n_neighbors=2)
    enn = EditedNearestNeighbours()
    adasyn = ADASYN(n_neighbors=2)
    smote = SMOTE(k_neighbors=2)
    smotetomek = SMOTETomek(smote=smote)
    smote_enn = SMOTEENN(smote=smote)

    imblearns = [nm1, enn, adasyn, smote, smotetomek, smote_enn]
    for imblearn in imblearns:
        try:
            X_im, y_im = imblearn.fit_resample(X, y)
            file.write(f"\n- MODELO DE RESAMPLING: {imblearn.__class__.__name__}\n")
            file.write("Distribution of class labels before resampling {}\n".format(Counter(y)))
            file.write("Distribution of class labels after resampling {}\n".format(Counter(y_im)))
            aplicar_modelos(X_im, y_im, file)
        except ValueError as e:
            file.write(f"Skipping {imblearn.__class__.__name__} due to ERROR: {e}\n")
        except RuntimeError as e:
            file.write(f"Skipping {imblearn.__class__.__name__} due to ERROR: {e}\n")

# dividir en test y train df_gestures
def split_df_gestures(df, target):
    test_df = df.loc[(df.SubjectID==101) | (df.SubjectID==105) | (df.SubjectID==201) | (df.SubjectID==202) | (df.SubjectID==301) | (df.SubjectID==302)]
    train_df = df.loc[(df.SubjectID!=101) & (df.SubjectID!=105) & (df.SubjectID!=201) & (df.SubjectID!=202) & (df.SubjectID!=301) & (df.SubjectID!=302)] 

    X_train = train_df.drop([target], axis=1)
    y_train=pd.DataFrame(train_df[target]) 
    X_test= test_df.drop([target], axis=1) 
    y_test =pd.DataFrame(test_df[target])

    # # Convertimos los df de target (y) a 1-d
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()

    return X_train, X_test, y_train, y_test

In [ ]:
gestures = df['GestureLabel'].unique()

with open('cvResults.txt', 'w') as file:
    for gesture in gestures:
        df_gesture = df[df['GestureLabel'] == gesture]
        #---------Dividimos en train y test------------
        X_train, X_test, y_train, y_test = split_df_gestures(df_gesture, 'CorrectLabel')  

        file.write(f'\n------------ CLASIFICACION GESTO {gesture} -----------\n')
        aplicar_imblearn(X_train, y_train, file)

El mejor es SMOTEtomek

In [ ]:
smotetomek = SMOTETomek(smote=SMOTE(k_neighbors=2))
X= df.drop(['CorrectLabel'], axis=1) 
y =pd.DataFrame(df['CorrectLabel'])
X_resampled, y_resampled = smotetomek.fit_resample(X, y)

# Combine the resampled features with the original SubjectID and GestureLabel
balanced_data = pd.DataFrame(X_resampled, columns=X.columns)
balanced_data['CorrectLabel'] = y_resampled

# Calculate the subject counts per gesture and execution status
subject_count_per_gesture_correct = balanced_data.groupby(["GestureLabel", "CorrectLabel"])["SubjectID"].nunique().reset_index()

# Plotting
plt.figure(figsize=(8, 5))
ax = sns.barplot(x="GestureLabel", y="SubjectID", hue="CorrectLabel",
                 data=subject_count_per_gesture_correct, palette="Paired")
# flare, Paired
ax.set_title("Distribución de los sujetos por gesto y estado de ejecución después de SMOTETomek")
ax.set_xlabel("Gesto")
ax.set_ylabel("Cantidad de sujetos")

# Set the x-axis ticks to the desired labels
gesture_labels = ['EFL', 'EFR', 'SFL', 'SFR', 'SAL', 'SAR', 'SFE', 'STL', 'STR']
ax.set_xticklabels(gesture_labels)

# Modify the legend labels
legend_labels = ['correctly executed', 'incorrectly executed', 'poorly executed']
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=legend_labels, title='Execution')
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

for i in ax.containers:
    ax.bar_label(i,)

plt.show()

## Pipeline con hiperparametrizacion y feature selection

In [7]:
# Para ignorar los FitFileWarning
warnings.filterwarnings("ignore", category=FitFailedWarning)

# definimos el param_grid
param_grid = [
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [2, 3, 5], 
        'classifier__weights': ['uniform', 'distance'], 
        'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [DecisionTreeClassifier()],
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__max_depth': [None, 10, 20, 30]
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [GaussianNB()]
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [SVC()],
        'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'classifier__C': [0.1, 1, 10, 100]
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [200, 300],
        'classifier__max_features': ['auto', 'sqrt', 'log2']
    },
    {
        'balance_data__smote__k_neighbors': [2, 3, 4],
        'reduce_dim__n_components': [0.5, 0.7, 0.8, 0.90, 1],
        'classifier': [LogisticRegression(max_iter=500)],
        'classifier__C': [0.01, 0.1, 1, 10, 100]
    }
]

logo = LeaveOneGroupOut()

for gesture in gestures:

    df_gesture = df[df['GestureLabel'] == gesture]

    #---------Dividimos en train y test------------
    X_train, X_test, y_train, y_test = split_df_gestures(df_gesture, 'CorrectLabel')   
    
    print(f'\n------------ CLASIFICACION GESTO {gesture} -----------\n')
    pipeline = Pipeline([('balance_data',  SMOTETomek(smote=SMOTE(k_neighbors=2))),
                        ('reduce_dim', PCA()),
                        ('classifier', KNeighborsClassifier(n_neighbors=2))])


    random = RandomizedSearchCV(pipeline, param_grid, cv=logo, n_jobs=-1)
    random.fit(X_train, y_train, groups=X_train['SubjectID'])
    best_pipeline = random.best_estimator_

    print('Training set score: ' + str(best_pipeline.score(X_train, y_train)))
    print('Test set score: ' + str(best_pipeline.score(X_test, y_test)))
    print(best_pipeline)

    globals()[f'pipeline_{gesture}'] = best_pipeline



KeyboardInterrupt: 

In [ ]:
pipeline_2.get_params().keys()
